In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython


In [2]:
%reload_ext Cython

In [3]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [4]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
import data_process, path
import data_generator
from data_functions import make_date_to_MYR, make_MYR_to_date
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from path import col_seq_can, col_seq_can_all
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

In [5]:
%config Completer.use_jedi = False

### Load Pmpm and IDs

In [6]:
# import boto3
# s3 = boto3.client('s3')
# bucket = 'cdphp-s3-us-e-p-pond'
# path_to_save = '../../../data/PMPM.p'
# s3_path = 'rpi/nneehal/DATA/pmpm_202108_processed_ready.p'
# success = s3.download_file(bucket, s3_path, path_to_save)

In [7]:
# path_to_save = '../../../data/ids.csv'
# s3_path = 'rpi/blind_matching_study/input_data/RPI Test Cohort.csv'
# success = s3.download_file(bucket, s3_path, path_to_save)

In [8]:
%%time
ids_csv = '../../../data/ids.csv'
pmpm_p = '../../../data/PMPM.p'

treated_ids = pd.read_csv(ids_csv)
pmpm = pd.read_pickle(pmpm_p)

#drop pmpm duplicates
pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)


CPU times: user 8.88 s, sys: 11 s, total: 19.9 s
Wall time: 19.9 s


In [9]:
treated_ids.shape

### Drop Unecessary data points

In [10]:
pmpm_backup = pmpm.copy()

In [11]:
pmpm_backup.MYR.min(), pmpm_backup.MYR.max()

(201701, 202108)

In [12]:
#pmpm_backup2 = pmpm_backup[pmpm_backup.MYR >= 202003].reset_index(drop = True)

In [13]:
pmpm_backup.shape

In [14]:
pmpm = pmpm[pmpm.MYR >= 202103].reset_index(drop = True)

In [15]:
pmpm.head()

PERS_ID     MYR MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT MEDICAID_LOB  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0      -3  202103        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1      -3  202104        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
2      -3  202105        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
3      -3  202106 

In [16]:
pmpm.MBR_GNDR.replace(to_replace = {'F':0, 'M': 1}, inplace = True)
# pmpm.MEDICAID_LOB.unique()

### Include Line of Bussiness Feature 1 Not medicaid 0 medicaid

In [17]:
# medicaid_mask = pmpm.MEDICAID_LOB.str.contains(r'.*Medicaid-.*')
# pmpm['LOB'] = 1
# pmpm.loc[medicaid_mask, 'LOB'] = 0

### Separate Treated Controls

In [18]:
treated_ids.iloc[0]

PERS_ID    2778075
Name: 0, dtype: int64

In [19]:
pmpm.PERS_ID = pmpm.PERS_ID.astype(int)

In [20]:
%%time
id_filter = pmpm.PERS_ID.isin(list(treated_ids.values[:,0]))
treated, controls = pmpm[id_filter].reset_index(drop = True), pmpm[~id_filter].reset_index(drop = True)

CPU times: user 499 ms, sys: 307 ms, total: 807 ms
Wall time: 806 ms


In [21]:
treated = treated[treated.MYR >= 202108].reset_index(drop = True)

# Take average of treated and controls

### Generate Treated

In [24]:
treated2 = data_process.get_average_record(df=treated, pmpm=pmpm_backup.copy())

PERS_ID
First date
One year before
history from original pmpm
record size calculated
record filtered
returning from f1...
f1 done


100%|██████████| 97/97 [00:00<00:00, 38287.92it/s]

f2 done
stacking done
returning..


In [25]:
treated2

PERS_ID     MYR AGE_AT_MIDMONTH    LOB MBR_GNDR ALLOW_AMT ALLOW_ER ALLOW_IP   CC_ADHD CC_ALZHEIMER CC_ALZHEIMER_PLUS    CC_AMI  CC_ANEMIA CC_ANXIETY_DISORDER CC_ARTHRITIS CC_ASTHMA CC_ATRIAL_FIB CC_AUTISM CC_BIPOLAR    CC_BPH CC_BREAST_CANCER CC_CATARACT CC_CEREBRAL_PALSY CC_CHRONIC_KIDNEY CC_COLORECTAL_CANCER   CC_COPD CC_CYSTIC_FIBROSIS CC_DEPRESSION CC_DEPRESSIVE_DISORDERS CC_DEVELOP_DELAYS CC_DIABETES CC_DISAB_DX_CNT CC_DX_CNT CC_ENDOMETRIAL_CANCER CC_EPILEPSY CC_FIBROMYALGIA CC_GLAUCOMA CC_HEARING_IMPAIR CC_HEART_FAILURE CC_HEPATITIS_A CC_HEPATITIS_B_ACUTE CC_HEPATITIS_B_CHRONIC CC_HEPATITIS_C_ACUTE CC_HEPATITIS_C_CHRONIC CC_HEPATITIS_C_UNSPECIFIED CC_HEPATITIS_D CC_HEPATITIS_E CC_HEPATITIS_GEN CC_HIP_FRACTURE CC_HYPERLIPIDEMIA CC_HYPERTENSION CC_HYPOTHYROID CC_INT_DISAB CC_ISCHEMIC CC_LEARN_DISAB CC_LEUKEMIAS  CC_LIVER CC_LUNG_CANCER CC_MIGRAINE CC_MOBILITY_IMPAIR CC_MULTIPLE_SCLEROSIS CC_MUSCULAR_DYSTROPHY CC_OBESITY CC_OSTEOPOROSIS CC_PERIPHERAL CC_PERSONALITY  \
0   2778075  202108         62.7692      0        0   2711.15        0  461.047         0            0                 0         0          0            0.538462            0         0             0         0          0         0                0           0                 0          0.307692                    0         0                  0      0.769231                       0                 0           1               0   4.38462                     0           0        0.307692           0                 0                0              0                    0                      0                    0                      0                          0              0              0                0               0          0.307692        0.307692              0            0           0              0            0  0.846154              0           0                  0                     0                     0          0               0             0              0   
1   2824337  202108         59.3077      0        1   2834.15        0  509.865         0            0                 0         0          0                   1            0         0             0         0   0.307692  0.307692                0    0.923077                 0                 1                    0         0                  0             1                       0                 0           1         1.15385   9.46154                     0           0        0.923077           0                 0                0              0                    0                      0                    0                      0                          0              0              0                0               0                 1               1              0            0           0              0            0         0              0           0           0.307692                     0                     0          0               0             0              0   
2   2826151  202108         59.1538      0        0   4332.31  328.983  1082.02         0            0                 1         0          0                   1            1         0             0         0          1         0                0           0                 0          0.692308                    0         1                  0             1                       1                 0           0               0   15.9231                     0           0               1    0.538462                 0                0              0                    0                      0                    0                      0                          0              0              0                0               0                 0               0              0            0           0              0            0         0              0           1                  0                     0                     0          1               0             0              1   
3   2851787  202108         57.3077      0  

In [139]:
treated2.to_pickle("jeremy_avg_treated.p")

### Generate Controls - Is it possible prior to matching??

In [151]:
controls2 = data_process.get_average_controls_jeremy(controls.copy(), pmpm_backup.copy())

[-12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1]
(158288, 92)
(158288, 77)
[-13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2]
(162237, 92)
(320525, 77)
[-14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3]
(165840, 92)
(486365, 77)
[-15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4]
(168384, 92)
(654749, 77)
[-16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5]
(169947, 92)
(824696, 77)
[-17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6]
(169841, 92)
(994537, 77)


In [142]:
controls2.to_pickle("jeremy_avg_controls.p")

### Save in S3 for reuse

In [150]:
# !aws s3 cp jeremy_avg_treated.p s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/Blind_Study_New/avg_treated.p --acl bucket-owner-full-control
# !aws s3 cp jeremy_avg_controls.p s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/Blind_Study_New/avg_controls.p --acl bucket-owner-full-control

## Match begin

In [37]:
#cols = list(col for col in treated2.columns if col not in ['PERS_ID', 'MYR'])
#columns_to_match = cols
columns_to_match = ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR','CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT', 'ACUTE', 'ACUTE2']

In [15]:
def get_data(data_struct, cont):
    cont = cont.copy()
    keys = list(data_struct.keys())
    dat1 = pd.DataFrame()
    for key in keys:
        dat1 = pd.concat([dat1, data_struct[key]], ignore_index = True)
    dat2 = dat1[['PERS_ID', 'MYR']]
    dat = pd.merge(dat2, cont, on = ['PERS_ID', 'MYR']).reset_index(drop = True)
    return dat

### General Configurations for All Matching Scenaria

In [16]:
#columns_to_match = columns_to_match + ['IP_BIN2', 'ER_BIN2']
#hier_cols = ['IP_BIN2', 'ER_BIN2', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6']
hier_cols = None #['CC_DX_CNT']
hier_range = [1]
exact_match_dict = {'ACUTE':None, 'ACUTE2':None}
match_on = [ 'ALLOW_AMT', 'CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT']
            


In [17]:
cols_to_keep = ["PERS_ID", "MYR", 'AGE_AT_MIDMONTH', 'MBR_GNDR']

## Load Model

In [18]:
import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
from tensorflow.keras import models, layers
import tensorflow.keras.backend as K

model = models.load_model("../../../temp/TF_Latent_Model_m6_Canary_stacked", compile=False)

/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type

In [19]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [20]:
%%time
#transform data of sick and healthy treated, and controls
tr2 = data_process.transform_features_with_NN(model, treated[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
cntrls = data_process.transform_features_with_NN(model, controls[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

CPU times: user 11.3 s, sys: 3.64 s, total: 14.9 s
Wall time: 8.52 s


## Scenario 1 Rigorous

In [21]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat1 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:220845 before they were:220845
CPU times: user 1min 38s, sys: 1min 32s, total: 3min 11s
Wall time: 14.1 s


In [22]:
dat1.shape

## Scenario 2 Non Rigorous

In [23]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat2 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:220845 before they were:220845
CPU times: user 1min 40s, sys: 1min 32s, total: 3min 12s
Wall time: 15.2 s


In [24]:
dat2.shape

### Scenario 3 Rigorous 6 months

In [25]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat3 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:220845 before they were:220845
CPU times: user 1min 41s, sys: 1min 31s, total: 3min 12s
Wall time: 14.9 s


In [26]:
dat3.MYR.min(), dat3.MYR.max()

(202103, 202108)

In [27]:
dat3.shape

## Scenario 4 Non Rigorous 6 Months

In [28]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat4 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:220845 before they were:220845
CPU times: user 1min 30s, sys: 1min 24s, total: 2min 55s
Wall time: 15.8 s


In [29]:
dat4.shape

## Scenario 5 Medicaid Rigorous 

In [30]:
controlsM = controls[controls.LOB == 0].reset_index(drop = True)

In [31]:
cntrlsM = data_process.transform_features_with_NN(model, controlsM[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

In [32]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat5 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:53312 before they were:53312
CPU times: user 36.6 s, sys: 29.2 s, total: 1min 5s
Wall time: 5.23 s


In [33]:
dat5.shape

## Scenario 6 Medicaid Non Rigorous

In [34]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat6 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:53312 before they were:53312
CPU times: user 38 s, sys: 34 s, total: 1min 12s
Wall time: 5.71 s


In [35]:
dat6.shape

### Scenario 7 Medicaid Rigorous 6 months

In [36]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat7 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:53312 before they were:53312
CPU times: user 38 s, sys: 32.5 s, total: 1min 10s
Wall time: 5.44 s


In [37]:
dat7.MYR.min(), dat7.MYR.max()

(202103, 202108)

In [38]:
dat7.shape

## Scenario 8 Medicaid Non Rigorous 6 Months

In [39]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat8 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:53312 before they were:53312
CPU times: user 36.1 s, sys: 32.6 s, total: 1min 8s
Wall time: 5.58 s


In [40]:
dat8.shape

# Save Stats

In [41]:
from utils import statistics_first_month
import scipy.stats as sts

In [42]:
columns_to_match = treated.columns.tolist()

In [43]:
%%time
tr_desc0, Ntr, treated_first_month = statistics_first_month(treated, columns_to_match, return_array = True)
tr_desc0 = tr_desc0.T

con1, N1, controls_first_month = statistics_first_month(dat1, columns_to_match, return_array = True)
con1 = con1.T

con2, N2, controls_first_month = statistics_first_month(dat2, columns_to_match, return_array = True)
con2 = con2.T

con3, N3, controls_first_month = statistics_first_month(dat3, columns_to_match, return_array = True)
con3 = con3.T

con4, N4, controls_first_month = statistics_first_month(dat4, columns_to_match, return_array = True)
con4 = con4.T

con5, N5, controls_first_month = statistics_first_month(dat5, columns_to_match, return_array = True)
con5 = con5.T

con6, N6, controls_first_month = statistics_first_month(dat6, columns_to_match, return_array = True)
con6 = con6.T

con7, N7, controls_first_month = statistics_first_month(dat7, columns_to_match, return_array = True)
con7 = con7.T

con8, N8, controls_first_month = statistics_first_month(dat8, columns_to_match, return_array = True)
con8 = con8.T

un_con_all, N9, controls_first_month =  statistics_first_month(controls, columns_to_match, return_array = True)
un_con_all = un_con_all.T

un_con_med, N10, controls_first_month =  statistics_first_month(controlsM, columns_to_match, return_array = True)
un_con_med = un_con_med.T



CPU times: user 1min 51s, sys: 1.65 s, total: 1min 53s
Wall time: 1min 53s


In [52]:
alli = pd.concat((tr_desc0[['mean','std']], un_con_all[['mean','std']], con1[['mean','std']], con2[['mean','std']], con3[['mean','std']], 
                  con4[['mean','std']], un_con_med[['mean','std']], con5[['mean','std']], con6[['mean','std']], con7[['mean','std']], con8[['mean','std']]),
                 axis = 1, keys = ['Treated', 'Unmatched Controls', 'Rig', 'NonRig', 'Rig6', 'NonRig6', 'Unmatched Med_Controls', 
                                   'Rig-Med', 'NonRig-Med', 'Rig-Med6','NonRig-Med6'])

In [50]:
pd.options.display.float_format = '{:.2f}'.format

In [58]:
alli[2:]

Treated         Unmatched Controls            Rig         NonRig           Rig6         NonRig6         Unmatched Med_Controls         Rig-Med         NonRig-Med         Rig-Med6         NonRig-Med6        
                              mean     std               mean     std   mean     std   mean     std   mean     std    mean     std                   mean     std    mean     std       mean     std     mean     std        mean     std
MBR_GNDR                      0.34    0.48               0.44    0.50   0.34    0.48   0.34    0.47   0.34    0.48    0.34    0.47                   0.40    0.49    0.34    0.48       0.34    0.47     0.34    0.47        0.34    0.48
AGE_AT_MIDMONTH              37.87   11.87              48.09   20.25  38.57   12.07  38.41   12.16  38.21   11.85   38.25   12.04                  34.04   16.91   38.43   12.03      38.29   12.20    38.26   11.97       38.21   12.21
ALLOW_AMT                   709.13 1425.41            1030.41 5321.78 784.93 1769.64 746.76 1675.38 815.46 1751.79  827.68 1812.99                 822.60 2931.48  764.03 1668.96     684.97 1516.78   791.23 1699.56      785.85 1658.51
ALLOW_ER                     22.19  110.86              20.08  192.30  24.81  165.07  20.67  136.74  15.77  117.10   18.41  131.77                  23.18  140.56   27.58  129.18      27.47  138.48    24.35  122.68       28.61  160.49
ALLOW_IP                    279.28 1205.83             249.13 4264.12  71.96  612.30  50.17  535.01  97.50  742.13   79.16  641.04                 192.86 2071.91  119.48  803.87      73.54  661.36   139.34  859.68      138.33  849.93
CC_ADHD                       0.23    0.42               0.04    0.20   0.08    0.27   0.09    0.29   0.09    0.28    0.09    0.28                   0.09    0.28    0.11    0.31       0.11    0.32     0.12    0.33        0.11    0.31
CC_ALZHEIMER                  0.01    0.09               0.01    0.07   0.00    0.00   0.00    0.00   0.00    0.00    0.00    0.00                   0.00    0.03    0.00    0.00       0.00    0.00     0.00    0.00        0.00    0.03
CC_ALZHEIMER_PLUS             0.04    0.19               0.02    0.13   0.01    0.07   0.01    0.08   0.00    0.04    0.00    0.07                   0.01    0.08    0.02    0.13       0.01    0.11     0.01    0.10        0.01    0.11
CC_AMI                        0.02    0.13               0.00    0.06   0.00    0.06   0.00    0.05   0.00    0.06    0.00    0.03                   0.00    0.05    0.01    0.07       0.01    0.08     0.00    0.06        0.00    0.05
CC_ANEMIA                     0.16    0.37               0.07    0.26   0.07    0.26   0.09    0.28   0.08    0.26    0.08    0.27                   0.07    0.26    0.11    0.31       0.11    0.31     0.10    0.30        0.11    0.31
CC_ANXIETY_DISORDER           0.90    0.30               0.22    0.42   0.40    0.49   0.41    0.49   0.39    0.49    0.39    0.49                   0.29    0.46    0.49    0.50       0.48    0.50     0.47    0.50        0.45    0.50
CC_ARTHRITIS                  0.14    0.35               0.14    0.34   0.10    0.30   0.11    0.31   0.10    0.30    0.11    0.31                   0.08    0.27    0.15    0.36       0.13    0.34     0.13    0.34        0.12    0.32
CC_ASTHMA                     0.21    0.41               0.06    0.23   0.08    0.27   0.09    0.29   0.08    0.28    0.09    0.28                   0.09    0.28    0.13    0.34       0.14    0.34     0.13    0.34        0.12    0.33
CC_ATRIAL_FIB                 0.04    0.19               0.03    0.17   0.01    0.07   0.01    0.10   0.02    0.13    0.01    0.10                   0.01    0.09    0.01    0.07       0.01    0.10     0.01    0.10        0.01    0.11
CC_AUTISM                     0.04    0.19               0.01    0.08   0.01    0.08   0.01    0.10   0.01    0.08    0.01    0.07                   0.02    0.13    0.01    0.12       0.02    0.13     0.01    0.11        0.01    0.11
CC_BIPOLAR                    0.41    0.49       

In [59]:
alli[2:].to_csv("./results/stats_mean_std/seq_1_8_mean_std.csv", header=True, index=True)

## Latent Stats

## Save latent features

In [57]:
dat_lists = [dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8]
file_id_start = 1
for data in dat_lists:
    dat_lat = data_process.transform_features_with_NN(model, data[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
    dat_lat.to_csv(f"./results/latent_features/seq_{file_id_start}.csv", index=False, header=True)
    file_id_start += 1

## Save IDs

In [87]:
ids = pd.concat([dat1.PERS_ID, dat2.PERS_ID, dat3.PERS_ID, dat4.PERS_ID, dat5.PERS_ID, dat6.PERS_ID, dat7.PERS_ID, dat8.PERS_ID], axis=1, ignore_index=True)
ids.rename(columns={0:"seq1", 1:"seq2", 2:"seq3", 3:"seq4", 4:"seq5", 5:"seq6", 6:"seq7", 7:"seq8"}, inplace=True)
ids.fillna(-1, inplace=True)
ids = ids.astype(int)

In [88]:
ids

seq1     seq2     seq3     seq4     seq5     seq6     seq7     seq8
0     5036526  5036526  2775576  2775576  5036526  5036526  2821732  2821732
1     2837861  2837861  4079774  4079774  2837861  2837861  2789472  2789472
2     3337693  3337693  2796878  2796878  3845280  3845280  2761276  2761276
3     2753776  2753776  4068657  4068657  3845273  3845273  2750362  2750362
4     5028034  5028034  3815531  3815531  2807827  2807827  4283124  4283124
...       ...      ...      ...      ...      ...      ...      ...      ...
2215       -1  2999197       -1  2928317       -1  2914427       -1  3710628
2216       -1  2926686       -1  2950349       -1  2920338       -1  2917279
2217       -1  4184576       -1  2940089       -1  2919825       -1  2917279
2218       -1  5034360       -1  2959448       -1  4246351       -1  2914053
2219       -1  2969452       -1  5060374       -1  5037823       -1  2994026

[2220 rows x 8 columns]

In [89]:
ids.to_csv("./results/ids/ids_seq_1_8.csv", index=True, header=True)

## Ratios with/without Std

In [316]:
disease_cols = [cols for cols in treated.columns if "CC" in cols and cols!='CC_DX_CNT']
other_binary_cols = ['MBR_GNDR', 'LOB', 'PREGNANCY', 'IP_BIN', 'ER_BIN', 'ACUTE']
binary_cols = other_binary_cols + disease_cols 
restricted_cols = ['PERS_ID', 'MYR', 'MEDICAID_LOB']
continuous_cols = [cols for cols in treated.columns if cols not in binary_cols + restricted_cols]

In [317]:
def calculate_continuous_d(t, c, std):
    Xt = np.mean(t)
    Xc = np.mean(c)
    St = np.std(t)
    Sc = np.std(c)
    if std==1: 
        d = np.abs((Xt-Xc)/(np.sqrt((St**2 + Sc**2)/2)))
    else:
        d = np.abs((Xt-Xc))
    return d

In [318]:
def calculate_binary_d(t, c, std):
    Pt = np.mean(t)
    Pc = np.mean(c)
    if std==1:
        d = np.abs((Pt-Pc)/(np.sqrt((Pt*(1-Pt)+Pc*(1-Pc))/2)))
    else:
        d = np.abs(Pt-Pc)
    return d

### 1.1 Whole Population as controls

In [319]:
std = 1

In [320]:
d_metric_before_all = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated[[col]].values, controls[[col]].values, std=std)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated[[col]].values, controls[[col]].values, std=std)
        d_metric_before_all.loc[0,col] = d

In [321]:
d_metric_after_all = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        i=0
        for dat_ in dat_lists[:4]:
            if col in binary_cols:
                d = calculate_binary_d(treated[[col]].values, dat_[[col]].values, std=std)
            elif col in continuous_cols:
                d = calculate_continuous_d(treated[[col]].values, dat_[[col]].values, std=std)
            d_metric_after_all.loc[i,col] = d
            i+=1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [322]:
d_metric_after_all

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.00             0.06       0.05      0.02      0.22     0.43          0.13               0.22    0.14       0.29                 1.24          0.13       0.38           0.22       0.20        0.79    0.07              0.02         0.18                nan               0.48                  0.02     0.45                0.09           1.07                     0.40               0.10         0.17             0.32       1.28                   0.10         0.24             0.50         0.04               0.16              0.36             nan                  0.06                     nan                   nan                    0.14                        0.19             nan             nan              0.20             0.06               0.08             0.32            0.05          0.08         0.20            0.19          0.06      0.45             nan         0.03                0.15                   0.17                   0.09        0.03             0.07           0.02   
1      0.00             0.05       0.04      0.01      0.24     0.37          0.13               0.21    0.16       0.24                 1.21          0.10       0.33           0.18       0.18        0.79    0.04              0.02         0.17               0.03               0.46                  0.08     0.42                0.05           1.00                     0.38               0.08         0.15             0.30       1.20                   0.12         0.24             0.44         0.03               0.22              0.34             nan                  0.04                     nan                   nan                    0.11                        0.17             nan             nan              0.17             0.03               0.08             0.27            0.01          0.09         0.21            0.13          0.07      0.38            0.06         0.01                0.16                   0.14                   0.05        0.02             0.08           0.05   
2      0.00             0.02       0.05      0.06      0.19     0.37          0.13               0.25    0.14       0.27                 1.24          0.15       0.36           0.11       0.22        0.81    0.06              0.04         0.19               0.06               0.52                  0.07     0.51                0.09           1.06                     0.38               0.13         0.24             0.44       1.29                   0.10         0.25             0.52         0.00               0.10              0.38             nan                   nan                     nan                  0.06                    0.19                        0.21             nan             nan              0.23              nan               0.01             0.21            0.04          0.12         0.18            0.14          0.12      0.42             nan         0.10                0.19                   0.09                    nan        0.04             0.07           0.02   
3  

In [323]:
d_metric_before_all

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.19             0.70       0.04      0.05      0.04     0.56          0.04               0.11    0.14       0.27                 1.82          0.01       0.45           0.01       0.21        1.01    0.16              0.08         0.39               0.04               0.42                  0.06     0.34                0.07           1.47                     0.45               0.10         0.12             0.32       1.23                   0.10         0.27             0.57         0.15               0.28              0.23            0.01                  0.03                    0.04                  0.01                    0.15                        0.21            0.01            0.00              0.22             0.06               0.20             0.02            0.01          0.15         0.07            0.15          0.12      0.41            0.09         0.18                0.15                   0.10                   0.03        0.11             0.10           0.21   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  LOB  PREGNANCY  IP_BIN  ER_BIN  ACUTE  IP_C  ER_C  ACUTE_C  IP_BIN6  ER_BIN6  ACUTE6  IP_BIN2  ER_BIN2  ACUTE2  
0            0.63         0.16                0.15     1.25              0.44                    0.53            0.16             0.05       0.04        1.23                0.09              0.05 2.58       0.04    0.29    0.22   0.35  1.11  1.13     1.31     1.03     0.88    1.29     0.59     0.56    0.77

In [324]:
ratio_1_4 = d_metric_after_all.values / d_metric_before_all.values
ratio_1_4 = pd.DataFrame(ratio_1_4, columns=d_metric_after_all.columns)

In [325]:
ratio_1_4

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.00             0.08       1.26      0.34      4.98     0.76          3.68               2.04    0.98       1.09                 0.68         10.09       0.86          17.34       0.96        0.78    0.44              0.23         0.47                nan               1.15                  0.33     1.34                1.18           0.73                     0.88               0.98         1.47             1.00       1.04                   1.01         0.90             0.87         0.28               0.58              1.54             nan                  2.25                     nan                   nan                    0.93                        0.91             nan             nan              0.91             1.07               0.39            13.37            5.23          0.53         2.96            1.31          0.50      1.09             nan         0.16                1.04                   1.72                   3.29        0.32             0.66           0.10   
1      0.00             0.07       1.18      0.20      5.43     0.66          3.68               1.96    1.11       0.89                 0.67          7.53       0.73          14.35       0.85        0.78    0.27              0.23         0.45               0.72               1.10                  1.34     1.24                0.72           0.68                     0.84               0.75         1.31             0.93       0.97                   1.19         0.89             0.77         0.19               0.79              1.48             nan                  1.59                     nan                   nan                    0.72                        0.80             nan             nan              0.77             0.53               0.41            11.23            0.54          0.59         3.10            0.88          0.62      0.92            0.67         0.06                1.07                   1.39                   2.01        0.14             0.80           0.22   
2      0.00             0.03       1.32      1.20      4.29     0.66          3.68               2.39    0.98       1.01                 0.68         11.46       0.80           8.92       1.05        0.80    0.41              0.54         0.50               1.44               1.24                  1.11     1.52                1.18           0.72                     0.83               1.34         1.99             1.37       1.05                   1.01         0.94             0.91         0.00               0.38              1.63             nan                   nan                     nan                  4.36                    1.28                        1.01             nan             nan              1.08              nan               0.05             8.89            4.15          0.79         2.73            0.96          1.01      1.01             nan         0.56                1.30                   0.86                    nan        0.39             0.66           0.09   
3  

In [326]:
mask_1_4 = ratio_1_4[ratio_1_4<=1.0]

In [327]:
mask_1_4

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.00             0.08        nan      0.34       nan     0.76           nan                nan    0.98        nan                 0.68           nan       0.86            nan       0.96        0.78    0.44              0.23         0.47                nan                nan                  0.33      nan                 nan           0.73                     0.88               0.98          nan             1.00        nan                    nan         0.90             0.87         0.28               0.58               nan             nan                   nan                     nan                   nan                    0.93                        0.91             nan             nan              0.91              nan               0.39              nan             nan          0.53          nan             nan          0.50       nan             nan         0.16                 nan                    nan                    nan        0.32             0.66           0.10   
1      0.00             0.07        nan      0.20       nan     0.66           nan                nan     nan       0.89                 0.67           nan       0.73            nan       0.85        0.78    0.27              0.23         0.45               0.72                nan                   nan      nan                0.72           0.68                     0.84               0.75          nan             0.93       0.97                    nan         0.89             0.77         0.19               0.79               nan             nan                   nan                     nan                   nan                    0.72                        0.80             nan             nan              0.77             0.53               0.41              nan            0.54          0.59          nan            0.88          0.62      0.92            0.67         0.06                 nan                    nan                    nan        0.14             0.80           0.22   
2      0.00             0.03        nan       nan       nan     0.66           nan                nan    0.98        nan                 0.68           nan       0.80            nan        nan        0.80    0.41              0.54         0.50                nan                nan                   nan      nan                 nan           0.72                     0.83                nan          nan              nan        nan                    nan         0.94             0.91         0.00               0.38               nan             nan                   nan                     nan                   nan                     nan                         nan             nan             nan               nan              nan               0.05              nan             nan          0.79          nan            0.96           nan       nan             nan         0.56                 nan                   0.86                    nan        0.39             0.66           0.09   
3  

features that got matched well for all combinations - standardarized absolute mean difference got reduced after matching ~ ratio <=1.0

In [328]:
mask_1_4.columns[~mask_1_4.isnull().any()].size

In [329]:
mask_1_4.columns[~mask_1_4.isnull().any()]

Index(['MBR_GNDR', 'AGE_AT_MIDMONTH', 'CC_ADHD', 'CC_ANXIETY_DISORDER', 'CC_ASTHMA', 'CC_BIPOLAR', 'CC_BPH', 'CC_BREAST_CANCER', 'CC_CATARACT', 'CC_DEPRESSION', 'CC_DEPRESSIVE_DISORDERS', 'CC_EPILEPSY', 'CC_FIBROMYALGIA', 'CC_GLAUCOMA', 'CC_HEARING_IMPAIR', 'CC_HYPERLIPIDEMIA', 'CC_INT_DISAB', 'CC_MIGRAINE', 'CC_OBESITY', 'CC_OSTEOPOROSIS', 'CC_PERIPHERAL', 'CC_PERSONALITY', 'CC_PROSTATE_CANCER', 'CC_PTSD', 'CC_SCHIZOPHRENIA', 'CC_SCHIZOPHRENIA_OTHER', 'CC_TOBACCO', 'LOB', 'IP_BIN', 'ER_BIN', 'ACUTE', 'IP_C', 'ER_C', 'ACUTE_C', 'IP_BIN6', 'ER_BIN6', 'ACUTE6', 'IP_BIN2', 'ER_BIN2', 'ACUTE2'], dtype='object')

features that didn't match for any combinations - standardarized absolute mean difference didn't got reduced after matching ~ ratio >=1.0

In [330]:
mask_1_4.columns[mask_1_4.isnull().all()].size

In [331]:
mask_1_4.columns[mask_1_4.isnull().all()]

Index(['ALLOW_AMT', 'ALLOW_IP', 'CC_ALZHEIMER', 'CC_ALZHEIMER_PLUS', 'CC_ARTHRITIS', 'CC_ATRIAL_FIB', 'CC_CHRONIC_KIDNEY', 'CC_COPD', 'CC_DIABETES', 'CC_ENDOMETRIAL_CANCER', 'CC_HEART_FAILURE', 'CC_HEPATITIS_A', 'CC_HEPATITIS_B_ACUTE', 'CC_HEPATITIS_C_ACUTE', 'CC_HEPATITIS_D', 'CC_HEPATITIS_E', 'CC_HYPERTENSION', 'CC_ISCHEMIC', 'CC_MOBILITY_IMPAIR', 'CC_MUSCULAR_DYSTROPHY', 'CC_STROKE', 'PREGNANCY'], dtype='object')

### 1.2 Only Medicaid Controls

In [332]:
d_metric_before_med = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated[[col]].values, controlsM[[col]].values, std=std)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated[[col]].values, controlsM[[col]].values, std=std)
        d_metric_before_med.loc[0,col] = d

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [333]:
d_metric_before_med

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.10             0.20       0.02      0.04      0.09     0.38          0.12               0.19    0.15       0.26                 1.49          0.17       0.33           0.18       0.12        0.81    0.01              0.04         0.20               0.05               0.52                  0.10     0.34                0.08           1.18                     0.40               0.04         0.22             0.27       1.30                   0.12         0.21             0.54         0.03               0.18              0.34            0.02                  0.04                    0.05                  0.02                    0.09                        0.17            0.01            0.01              0.15             0.04               0.13             0.28            0.11          0.09         0.18            0.08          0.08      0.42            0.06         0.12                0.15                   0.09                   0.03        0.12             0.05           0.07   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  LOB  PREGNANCY  IP_BIN  ER_BIN  ACUTE  IP_C  ER_C  ACUTE_C  IP_BIN6  ER_BIN6  ACUTE6  IP_BIN2  ER_BIN2  ACUTE2  
0            0.56         0.14                0.06     1.06              0.35                    0.42            0.17             0.07       0.08        0.89                0.08              0.05  nan       0.05    0.26    0.09   0.23  1.05  0.81     1.04     0.97     0.67    1.07     0.54     0.41    0.61

In [334]:
d_metric_after_med = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        i=0
        for dat_ in dat_lists[4:]:
            if col in binary_cols:
                d = calculate_binary_d(treated[[col]].values, dat_[[col]].values, std=std)
            elif col in continuous_cols:
                d = calculate_continuous_d(treated[[col]].values, dat_[[col]].values, std=std)
            d_metric_after_med.loc[i,col] = d
            i+=1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [335]:
d_metric_after_med

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.00             0.05       0.03      0.04      0.16     0.33          0.13               0.12    0.12       0.16                 0.99          0.04       0.20           0.22       0.14        0.58    0.04              0.04         0.23                nan               0.41                  0.07     0.22                 nan           0.76                     0.33               0.02         0.07             0.13       0.82                   0.13         0.11             0.32         0.05               0.22              0.27             nan                  0.09                     nan                   nan                    0.04                        0.04             nan             nan              0.04             0.06               0.07             0.18            0.06          0.01         0.09            0.12          0.10      0.38            0.09         0.04                0.07                   0.16                   0.06        0.11             0.07           0.11   
1      0.00             0.04       0.02      0.05      0.20     0.31          0.13               0.16    0.10       0.15                 1.02          0.03       0.18           0.18       0.13        0.60    0.03              0.04         0.18               0.06               0.37                  0.07     0.29                0.05           0.80                     0.37               0.06         0.08             0.17       0.89                   0.13         0.13             0.34         0.00               0.21              0.29            0.04                  0.04                    0.06                   nan                    0.04                        0.10             nan             nan              0.07             0.03               0.06             0.17            0.00          0.05         0.13            0.10          0.10      0.34            0.06         0.03                0.10                   0.15                   0.04        0.03             0.06           0.11   
2      0.00             0.01       0.04      0.01      0.14     0.25          0.13               0.17    0.14       0.19                 1.05          0.03       0.21           0.18       0.14        0.71    0.04              0.04         0.15               0.09               0.49                  0.07     0.36                0.09           0.82                     0.34               0.13         0.17             0.33       1.03                   0.13         0.21             0.44         0.06               0.16              0.34             nan                  0.09                    0.06                  0.06                    0.08                        0.17             nan             nan              0.11              nan               0.08             0.20            0.01          0.10         0.18            0.19          0.06      0.43            0.06         0.09                0.15                   0.09                    nan        0.06             0.04           0.02   
3  

In [336]:
ratio_5_8 = d_metric_after_med.values / d_metric_before_med.values
ratio_5_8 = pd.DataFrame(ratio_5_8, columns=d_metric_after_med.columns)

In [337]:
ratio_5_8

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.00             0.24       1.89      1.11      1.66     0.87          1.14               0.65    0.78       0.61                 0.67          0.21       0.60           1.18       1.14        0.72    4.97              0.97         1.14                nan               0.79                  0.66     0.65                 nan           0.64                     0.82               0.47         0.30             0.47       0.63                   1.10         0.50             0.59         1.77               1.19              0.81             nan                  2.32                     nan                   nan                    0.47                        0.21             nan             nan              0.27             1.55               0.51             0.64            0.54          0.16         0.51            1.43          1.23      0.89            1.35         0.31                0.51                   1.77                   1.98        0.91             1.35           1.59   
1      0.00             0.20       1.31      1.18      2.06     0.82          1.14               0.81    0.67       0.58                 0.69          0.19       0.54           1.00       1.06        0.74    3.61              0.97         0.93               1.13               0.72                  0.66     0.85                0.69           0.67                     0.93               1.29         0.39             0.63       0.69                   1.10         0.62             0.63         0.09               1.17              0.87            2.30                  1.16                    1.19                   nan                    0.39                        0.60             nan             nan              0.44             0.77               0.49             0.61            0.00          0.56         0.71            1.24          1.23      0.79            0.95         0.27                0.68                   1.67                   1.40        0.28             1.22           1.51   
2      0.00             0.07       2.22      0.20      1.50     0.66          1.14               0.87    0.92       0.72                 0.70          0.15       0.63           1.00       1.14        0.88    4.97              0.97         0.74               1.60               0.96                  0.66     1.05                1.13           0.69                     0.84               3.15         0.77             1.20       0.79                   1.10         0.98             0.82         2.08               0.87              1.00             nan                  2.32                    1.19                  2.72                    0.83                        1.03             nan             nan              0.74              nan               0.62             0.72            0.06          1.14         1.02            2.34          0.71      1.02            0.95         0.74                1.05                   0.94                    nan        0.52             0.84           0.25   
3  

In [338]:
mask_5_8 = ratio_5_8[ratio_5_8<=1.0]

In [339]:
mask_5_8

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.00             0.24        nan       nan       nan     0.87           nan               0.65    0.78       0.61                 0.67          0.21       0.60            nan        nan        0.72     nan              0.97          nan                nan               0.79                  0.66     0.65                 nan           0.64                     0.82               0.47         0.30             0.47       0.63                    nan         0.50             0.59          nan                nan              0.81             nan                   nan                     nan                   nan                    0.47                        0.21             nan             nan              0.27              nan               0.51             0.64            0.54          0.16         0.51             nan           nan      0.89             nan         0.31                0.51                    nan                    nan        0.91              nan            nan   
1      0.00             0.20        nan       nan       nan     0.82           nan               0.81    0.67       0.58                 0.69          0.19       0.54           1.00        nan        0.74     nan              0.97         0.93                nan               0.72                  0.66     0.85                0.69           0.67                     0.93                nan         0.39             0.63       0.69                    nan         0.62             0.63         0.09                nan              0.87             nan                   nan                     nan                   nan                    0.39                        0.60             nan             nan              0.44             0.77               0.49             0.61            0.00          0.56         0.71             nan           nan      0.79            0.95         0.27                0.68                    nan                    nan        0.28              nan            nan   
2      0.00             0.07        nan      0.20       nan     0.66           nan               0.87    0.92       0.72                 0.70          0.15       0.63           1.00        nan        0.88     nan              0.97         0.74                nan               0.96                  0.66      nan                 nan           0.69                     0.84                nan         0.77              nan       0.79                    nan         0.98             0.82          nan               0.87               nan             nan                   nan                     nan                   nan                    0.83                         nan             nan             nan              0.74              nan               0.62             0.72            0.06           nan          nan             nan          0.71       nan            0.95         0.74                 nan                   0.94                    nan        0.52             0.84           0.25   
3  

features that got matched well for all combinations - standardarized absolute mean difference got reduced after matching ~ ratio <=1.0

In [340]:
mask_5_8.columns[~mask_5_8.isnull().any()].size

In [341]:
mask_5_8.columns[~mask_5_8.isnull().any()]

Index(['MBR_GNDR', 'AGE_AT_MIDMONTH', 'CC_ADHD', 'CC_ALZHEIMER_PLUS', 'CC_ANEMIA', 'CC_ANXIETY_DISORDER', 'CC_ARTHRITIS', 'CC_ASTHMA', 'CC_BIPOLAR', 'CC_BREAST_CANCER', 'CC_CHRONIC_KIDNEY', 'CC_COLORECTAL_CANCER', 'CC_DEPRESSION', 'CC_DEPRESSIVE_DISORDERS', 'CC_DIABETES', 'CC_DX_CNT', 'CC_EPILEPSY', 'CC_FIBROMYALGIA', 'CC_HEPATITIS_C_CHRONIC', 'CC_HEPATITIS_GEN', 'CC_HYPERLIPIDEMIA', 'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_MIGRAINE', 'CC_OBESITY', 'CC_PERSONALITY', 'CC_PTSD', 'CC_SCHIZOPHRENIA', 'CC_SCHIZOPHRENIA_OTHER', 'CC_TOBACCO', 'IP_BIN', 'ER_BIN', 'ACUTE', 'IP_C', 'ER_C', 'ACUTE_C', 'IP_BIN6', 'ER_BIN6', 'ACUTE6', 'IP_BIN2', 'ER_BIN2', 'ACUTE2'], dtype='object')

features that didn't match for any combinations - standardarized absolute mean difference didn't got reduced after matching ~ ratio >=1.0

In [342]:
mask_5_8.columns[mask_5_8.isnull().all()].size

In [343]:
mask_5_8.columns[mask_5_8.isnull().all()]

Index(['ALLOW_AMT', 'ALLOW_IP', 'CC_AUTISM', 'CC_BPH', 'CC_CEREBRAL_PALSY', 'CC_HEPATITIS_A', 'CC_HEPATITIS_B_ACUTE', 'CC_HEPATITIS_B_CHRONIC', 'CC_HEPATITIS_C_ACUTE', 'CC_HEPATITIS_D', 'CC_HEPATITIS_E', 'CC_LEARN_DISAB', 'CC_MUSCULAR_DYSTROPHY', 'CC_VISUAL_IMPAIR', 'LOB', 'PREGNANCY'], dtype='object')

### 1.3 Merge and Save ratio

In [344]:
final_ratio = pd.concat([ratio_1_4, ratio_5_8], axis=0, ignore_index=True)
final_ratio

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.00             0.08       1.26      0.34      4.98     0.76          3.68               2.04    0.98       1.09                 0.68         10.09       0.86          17.34       0.96        0.78    0.44              0.23         0.47                nan               1.15                  0.33     1.34                1.18           0.73                     0.88               0.98         1.47             1.00       1.04                   1.01         0.90             0.87         0.28               0.58              1.54             nan                  2.25                     nan                   nan                    0.93                        0.91             nan             nan              0.91             1.07               0.39            13.37            5.23          0.53         2.96            1.31          0.50      1.09             nan         0.16                1.04                   1.72                   3.29        0.32             0.66           0.10   
1      0.00             0.07       1.18      0.20      5.43     0.66          3.68               1.96    1.11       0.89                 0.67          7.53       0.73          14.35       0.85        0.78    0.27              0.23         0.45               0.72               1.10                  1.34     1.24                0.72           0.68                     0.84               0.75         1.31             0.93       0.97                   1.19         0.89             0.77         0.19               0.79              1.48             nan                  1.59                     nan                   nan                    0.72                        0.80             nan             nan              0.77             0.53               0.41            11.23            0.54          0.59         3.10            0.88          0.62      0.92            0.67         0.06                1.07                   1.39                   2.01        0.14             0.80           0.22   
2      0.00             0.03       1.32      1.20      4.29     0.66          3.68               2.39    0.98       1.01                 0.68         11.46       0.80           8.92       1.05        0.80    0.41              0.54         0.50               1.44               1.24                  1.11     1.52                1.18           0.72                     0.83               1.34         1.99             1.37       1.05                   1.01         0.94             0.91         0.00               0.38              1.63             nan                   nan                     nan                  4.36                    1.28                        1.01             nan             nan              1.08              nan               0.05             8.89            4.15          0.79         2.73            0.96          1.01      1.01             nan         0.56                1.30                   0.86                    nan        0.39             0.66           0.09   
3  

In [345]:
final_ratio.to_csv("./results/ratio/ratio_with_std_1_8.csv", index=False, header=True)